### 단어\_태그\_T/F(받침여부)_원단어의발음
### https://docs.google.com/spreadsheets/d/1-9blXKjtjeKZqsf4NzHeYJCrr49-nXeRF6D80udfcwY/edit#gid=4
# 명사 NNG, 동사 VV, 형용사 VA


In [3]:
from konlpy.tag import Mecab
import pickle
import re
import sys
from gensim.models import TfidfModel
from gensim.models import Word2Vec
from gensim.corpora import Dictionary
from pprint import pprint
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import mglearn
from pprint import pprint
import numpy as np
import gc
import copy
import pandas as pd


class SB_Word2Vec():    
    
    def __init__(self, morph_list):
        self.dct = Dictionary(morph_list)
        self.corpus = [self.dct.doc2bow(line) for line in morph_list]
        self.build_Word2Vec(morph_list)
    
    def make_Word2Vec(self, morph_list, size=50, window=2, min_count=10, iteration=100):
        self.em = Word2Vec(morph_list, size=size, window=window, min_count=min_count, iter=iteration)
        self.em_vocab = list(self.em.wv.vocab.keys())
        self.em_vocab_dic = {word:idx for idx, word in enumerate(self.em_vocab)}

    def make_Word2Sen_matrix(self): 
        vocab_size = len(self.em_vocab)
        self.sen_matrix = np.zeros((len(self.corpus), vocab_size))
        for idx, row in enumerate(self.sen_matrix):
            for idx2, frequency in self.corpus[idx]:
                    if self.dct[idx2] in self.em_vocab:
                        self.sen_matrix[idx][self.em_vocab_dic[self.dct[idx2]]] = frequency                
        self.sim_matrix = np.zeros((vocab_size, vocab_size))
        for idx, w1 in enumerate(self.em_vocab):
            for idx2, w2 in enumerate(self.em_vocab):
                self.sim_matrix[idx][idx2] =  self.em.wv.similarity(w1, w2)

        self.word2sen_matrix = np.dot(self.sim_matrix, np.transpose(self.sen_matrix))

        return self.word2sen_matrix

    def get_sim_sen(self, keyword, main_text, number=1):
        self.sim_sen_index = np.argsort(self.word2sen_matrix[self.em_vocab_dic[keyword]])
        self.most_sim_sen_index = np.argmax(self.word2sen_matrix[self.em_vocab_dic[keyword]])
        index_list = self.sim_sen_index.reshape((-1,)).tolist()
        index_list.reverse()
        
        for idx, i in enumerate(index_list[:number]):
            print(str(idx + 1))
            print(main_text[i])
        return index_list
    
    def build_Word2Vec(self, morph_list):
        self.make_Word2Vec(morph_list)
        self.make_Word2Sen_matrix()
        
        
class SB_LDA():

    def make_lda(self, morph_joined, ntopic=10, learning_method='batch', max_iter=25, random_state=0, n_words=20):        
        self.vect = CountVectorizer(max_features=10000, max_df=.15)
        self.X = self.vect.fit_transform(morph_joined)
        self.lda = LatentDirichletAllocation(n_components=ntopic, learning_method=learning_method, max_iter=max_iter, random_state=random_state)
        self.document_topics = self.lda.fit_transform(self.X)
        self.sorting = np.argsort(self.lda.components_, axis=1)[:, ::-1]
        self.feature_names = np.array(self.vect.get_feature_names())
        mglearn.tools.print_topics(topics=range(ntopic), feature_names=self.feature_names, sorting=self.sorting, topics_per_chunk=5, n_words=n_words)

    def related_doc(self, main_text_list, topic_index, number=10):
        category = np.argsort(self.document_topics[:, topic_index])[::-1]
        related_docs = []
        for i in category[:number]:
            print(i)
            print(main_text_list[i] + ".\n")
            related_docs.append((i, main_text_list[i]))
        return related_docs

class SB_Tfidf():    
    
    def __init__(self, list_morph_merged):
        self.list_morph_merged = list_morph_merged
        self.dct = Dictionary(self.list_morph_merged)
        self.corpus = [self.dct.doc2bow(line) for line in self.list_morph_merged]

    def get_tfidf(self):       
        self.model = TfidfModel(self.corpus)
        self.tfidf = []
        for i in self.corpus:
             self.tfidf.append(sorted(self.model[i], key = lambda x: x[1], reverse=True))
        self.tfidf_hangul = []
        for idx1, i in enumerate(self.tfidf):
            self.tfidf_hangul.append([(self.dct[j[0]], j[1]) for j in i])        
        
        return self.tfidf_hangul
    
def frequency(merged):
    word_count = Counter(merged)
    word_count2 = []
    for i in word_count:
        word_count2.append((i, word_count[i]))
    word_count2 = sorted(word_count2, key=lambda x: x[1], reverse = True)
    return word_count2




class Social_analysis():
    
    non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
    syn_dic = {}
    theme_dic = {}
    del_list = []
    ngram_dic = {}
    exception_list=['맛', '밥', '물', '몸', '없', '있', '싫', '달', '굳', '굿', '속']

    default_dic_path = 'Data/custom_dic.csv'
    replace_dic = 'Data/replace_dic.csv'
    
    def __init__(self, corp_name):
        self.corp_name = corp_name
        self.mecab = Mecab()
        try:
            self.load_dictionary()
        except Exception as e:
            print('dictionary error\n', e)
            
    def load_dictionary(self):
        path = self.default_dic_path
        self.dic_df = pd.read_csv(path, encoding='cp949')
        self.dic_dif = self.dic_df.astype(str)
        for i in range(len(self.dic_df)):
            key = self.dic_df.loc[i,'key']
            value = self.dic_df.loc[i, 'value']
            syn = self.dic_df.loc[i, 'syn']
            theme = self.dic_df.loc[i, 'theme']

            if pd.isna(value):
                print('Need key & value')
                return
            self.ngram_dic[key] = value
            
            if not pd.isna(syn):
                self.syn_dic[value.split('_')[0]] = syn
            
            if not pd.isna(theme):
                self.theme_dic[value.split('_')[0]] = theme
            
    def DB_to_table(self, DBname='intake', keyword='intake'):
        import pymssql
        import pandas.io.sql as pdsql
        import pandas as pd
        self.query = \
        """
        SELECT user_id, created_at, main_text, hashtags, comments, likes, current_url FROM instaPosting WHERE keyword = '{}'
        """.format(keyword)
        conn = pymssql.connect("intakedb.c63elkxbiwfc.us-east-2.rds.amazonaws.com:1433", "gh", "ghintake", DBname)
        self.df = pdsql.read_sql_query(self.query, con=conn)
        # df['main_text'] = df.main_text.apply(lambda x: x.replace('#',' ').translate(self.non_bmp_map))
        # df['created_at'] = df.created_at.apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
        conn.close()       

    def insta_DB_to_table(self, DBname='intake', keyword='intake'):
        import pymssql
        import pandas.io.sql as pdsql
        import pandas as pd
        self.query = \
        """
        SELECT user_id, created_at, main_text, hashtags, comments, likes, current_url FROM instaPosting WHERE keyword = '{}'

        """.format(keyword)
        
        conn = pymssql.connect("intakedb.c63elkxbiwfc.us-east-2.rds.amazonaws.com:1433", "gh", "ghintake", DBname)
        self.df = pdsql.read_sql_query(self.query, con=conn)
        print(self.df)
        # df['main_text'] = df.main_text.apply(lambda x: x.replace('#',' ').translate(self.non_bmp_map))
        # df['created_at'] = df.created_at.apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
        conn.close()
    
    def shop_DB_to_table(self, DBname='intake', keyword='intake'):
        import pymssql
        import pandas.io.sql as pdsql
        import pandas as pd
        self.query = \
        """
        SELECT mall, keyword, productCode, date, user_id, productScore, recommScore, main_text FROM VproductReview WHERE keyword = '{}'
        """.format(keyword)
        conn = pymssql.connect("intakedb.c63elkxbiwfc.us-east-2.rds.amazonaws.com:1433", "gh", "ghintake", DBname)
        self.df = pdsql.read_sql_query(self.query, con=conn)
        # df['main_text'] = df.main_text.apply(lambda x: x.replace('#',' ').translate(self.non_bmp_map))
        # df['created_at'] = df.created_at.apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
        conn.close()
              
        
    def blog_DB_to_table(self, DBname='intake', keyword='intake'):
        import pymssql
        import pandas.io.sql as pdsql
        import pandas as pd
        self.query = \
        """
        SELECT keyword, created_at, post_name, main_text, current_url FROM NaverBlogReview WHERE keyword = '{}'
        """.format(keyword)
        conn = pymssql.connect("intakedb.c63elkxbiwfc.us-east-2.rds.amazonaws.com:1433", "gh", "ghintake", DBname)
        self.df = pdsql.read_sql_query(self.query, con=conn)
        # df['main_text'] = df.main_text.apply(lambda x: x.replace('#',' ').translate(self.non_bmp_map))
        # df['created_at'] = df.created_at.apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
        conn.close()
              
    
    def pickle_to_table(self, filename, columns=['user_id', 'created_at', 'main_text', 'hashtags', 'comments', 'likes', 'current_url']):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        data = data[1:]
        for idx, i in enumerate(data):
            data[idx][2] = i[2].replace('#', ' ').translate(self.non_bmp_map)
            data[idx][3] = '/'.join(i[3])
            data[idx][4] = '/'.join(i[4])
        self.df = pd.DataFrame(np.array(data), columns=['user_id', 'created_at', 'main_text', 'hashtags', 'comments', 'likes', 'current_url'])

  
    def hashtags_split(self, hashtags):        
        hashtags_split = []
        for i in hashtags:
            hashtags_split.append(i.split('/'))
        
        hashtags_list = []
        
        for i in hashtags_split:
            temp = []
            for j in i:
                if self.isHangul(j):
                    t_hashtags = j.translate(self.non_bmp_map)
                    temp.append(t_hashtags)
            hashtags_list.append(temp)
        self.hashtags_list = hashtags_list
        
        return hashtags_list

    
    def add_dictionary(self, *tokenized_list):
        origin_df = 1
        try:
            origin_df = pd.read_csv("C:\\mecab\\user-dic\\intake_dic.csv", encoding='utf-8', header=None)
        except:
            print('No default intake_dic')
        keyword_list = []
        for i in tokenized_list:
            if type(i) == list:
                for j in i:
                    j = j.split('_')
                    temp = [j[0],'' ,'' ,'' ,j[1],'*',j[2], j[3],'*','*','*','*','*']
                    keyword_list.append(temp)
            else:
                i = i.split('_')
                temp = [i[0],'','','',i[1],'*',i[2], i[3], '*','*','*','*','*']
                keyword_list.append(temp)


        keyword_df = pd.DataFrame(keyword_list)
        print(type(origin_df))
        if type(origin_df) != int:
            keyword_df = pd.concat((origin_df, keyword_df), ignore_index=True)
        else: 
            print('a')
            pass
        print(keyword_df.shape)

        keyword_df.to_csv("C:\\mecab\\user-dic\\intake_dic.csv", encoding='utf-8',index=None, header=False)    

        
    def ngram(self, parsed_list):
        ngram_list = []        
        adjustment = 0
        # 단어_tag의 리스트

        for idx in range(len(parsed_list)):
            idx2 = idx + adjustment

            if (idx2+self.ngram_size) > (len(parsed_list)):
                ngram_list.extend(parsed_list[idx2:])
                break
            n_filter = tuple(parsed_list[idx2: idx2 + self.ngram_size])
            key = ''.join([k.split('_')[0] for k in n_filter])
            if key in self.ngram_dic:
                ngram_list.append(self.ngram_dic[key])
                adjustment += (self.ngram_size - 1)
            else:
                ngram_list.append(n_filter[0])

        if self.ngram_size <= 1:
            return ngram_list
        else:
            self.ngram_size -= 1
            return self.ngram(ngram_list)       
        
    def morph_pos(self, text_list,  mode='list'):
        
        morph_list = []
        
        for j in text_list:
            parsed = self.mecab.pos(j)
            temp = []
            for i in parsed:
                if self.isHangul(i[0]):
                    temp.append('{}_{}'.format(i[0], i[1]))
                else: pass#print('{} 한글이 아님.'.format(i[0]))

            self.ngram_size = 6
            morph_list.append(self.ngram(temp))
            
        self.df['morph_list'] = morph_list
        
        return morph_list


    def set_with_order(self, sequence):
        seen = set()
        result = [x for x in sequence if not (x in seen or seen.add(x))]
        return result
        
 
    def filter_words(self, parsed_list, mode='syn'):
        # 1차원 리스트를 받음.

        deleted_list = [f for f in filter(lambda x: x not in self.del_list, parsed_list)]

        if mode == None:
            return deleted_list
        
        elif mode == 'syn':
            syn_list = [f.format(self.corp_name) for f in map(lambda x: self.syn_dic.get(x, x), deleted_list)]
            return syn_list
        
        elif mode == 'theme':
            theme_list = [f.format(self.corp_name) for f in map(lambda x: self.theme_dic.get(x, x), deleted_list)]
            return theme_list 
        
        else:
            return deleted_list
    
    def pos_extractor(self, parsed, mode = 'list', degree = 'syn'):
        
        noun_list = []
        adj_list = []
        verb_list = []
        nav_list = []
        total_list = [nav_list, noun_list, adj_list, verb_list]
        
        for j in parsed:
            nav_temp = []
            n_temp = []
            adj_temp = []
            verb_temp = []
            temp_list = [nav_temp,  n_temp, adj_temp, verb_temp]
            
            for i in j:
                i = i.split('_')
                if self.isHangul(i[0]):
                    if (len(i[0]) > 1) or (i[0] in self.exception_list):                        
                        if 'NN' in i[1]:
                            n_temp.append(i[0])
                            nav_temp.append(i[0])
                        elif 'VV'in i[1]:
                            adj_temp.append(i[0])
                            nav_temp.append(i[0])
                        elif 'VA' in i[1]:
                            verb_temp.append(i[0])
                            nav_temp.append(i[0])
                    else: pass
                        #print('{} 제외'.format(i[0]))
                else: pass#print('{} 한글이 아님.'.format(i[0]))

            
            for idx, li in enumerate(total_list):
                if mode == 'list':
                    li.append(self.filter_words(temp_list[idx], degree))
                elif mode == 'set':
                    li.append(self.set_with_order(self.filter_words(temp_list[idx], degree)))
                else:
                    print('Check mode')
                    return
            
            
        columns=['nav_list', 'noun_list', 'adj_list', 'verb_list']
        for i in  zip(columns, total_list):
            self.df[i[0]] = i[1]
            
        #return nav_list, noun_list, adj_list, verb_list # tuple(map(lambda x: [j.split('_')[0] for j in x], [nav_list, noun_list, adj_list, verb_list]))
    def merge_list(self, tokenized_list):
        return [j for i in tokenized_list for j in i]

    
    def join_list(self, tokenized_list):
        joined_list = []
        for idx, i in enumerate(tokenized_list):
            joined_list.append(" ".join(i))
        return joined_list
 
    def split_list(self, untokenized_list):
        hashtag_splited = []
        for idx, i in enumerate(untokenized):
            hashtag_splited.append(i.split('/'))
            return hastag_splited
        
    '''    
    def join_underbar(self, morph_list):

        all_list = []
        post_list=[]
        for i in morph_list:
            for j in i:
                post_list.append(j[0]+'_'+j[1])
            all_list.append([(' , ').join(post_list)])
            post_list=[] 
        all_list=np.array(all_list)
        
        return all_list'''

    def word_substitute(self, dataset, sublist):
        dataset = copy.deepcopy(dataset)
        sub_book = dict()
        for i in sublist:
            for j in i['sub_words']:
                sub_book[j] = i['main']
        gc.collect()
        for n, i in enumerate(dataset):
            dataset[n] = [sub_book.get(item,item) for item in i]

        del sub_book
        gc.collect()

        return dataset
    
    def word_delete(self, dataset, del_list):
        dataset = copy.deepcopy(dataset)

        for n, line in enumerate(dataset):
             dataset[n] = [i for i in line if i not in del_list]

        return dataset

    
    def isHangul(self, text):
        encText = text
        hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', encText))
        return hanCount > 0
    
    def convert_list(self, *tokenized_list):
        input_length = len(tokenized_list)
        lists = [[] for i in range(input_length)]

        for idx, li in enumerate(tokenized_list):
            for j in li:
                lists[idx].append(['/'.join(j)])

        converted_array = np.array(lists[0])
        for idx in range(input_length):
            try:
                converted_array = np.concatenate((converted_array, lists[idx + 1]), axis=1)
            except Exception as e:
                print(e,'끝')

        return converted_array

    def make_df(self, start_array, converted_array, end_array, columns=['user_id', 'created_at', 'main_text', 'morph_list', 'nav_list', 'noun_list', 'adj_list', 'verb_list', 'hashtags', 'comments', 'likes', 'current_url']):         
        df = pd.DataFrame(np.hstack((start_array, converted_array, end_array)), index=None, columns=columns)
        return df
    
    # 키워드 리스트 중 하나라도 있는 경우
    def word_check_or(self, text, keywords):
        if any(word in text for word in keywords):
            return 1
        else: return 0

    # 키워드 리스트에 있는 단어가 모두 있는 경우
    def word_check_and(self, text, keywords):
        if all(word in text for word in keywords):
            return 1
        else:
            return 0


    def word_check(self, method, keywords, df, column_name = 'main_text',filter_TF=True):
        
        filter_TF = 1 if filter_TF == True else 0
        if method == 'and':
            df['flags'] = df[column_name].apply(lambda x: self.word_check_and(x, keywords))
            return df.loc[df['flags'] == filter_TF]

        elif method == 'or':
            df['flags'] = df[column_name].apply(lambda x: self.word_check_or(x, keywords))
            return df.loc[df['flags'] == filter_TF]
        
        else:
            print('Select method, and/or')
            
    def df_str(df, *column_names):
        for i in column_names:
            df.loc[:,i] = df.loc[:,i].str.join('/')
        

In [4]:
itkinsta = Social_analysis('인테이크')
itkshop = Social_analysis('인테이크')
itkblog = Social_analysis('인테이크')
lnsinsta = Social_analysis('랩노쉬')
lnsshop = Social_analysis('랩노쉬')
lnsblog = Social_analysis('랩노쉬')
pckinsta = Social_analysis('피코크')
pckshop = Social_analysis('피코크')
pckblog = Social_analysis('피코크')
kgcinsta = Social_analysis('정관장')
kgcshop = Social_analysis('정관장')
kgcblog = Social_analysis('정관장')


In [5]:
itkinsta.insta_DB_to_table(DBname = 'intake', keyword = 'intake')
itkshop.shop_DB_to_table('intake', 'intake')
itkblog.blog_DB_to_table('intake', 'intake')
lnsinsta.insta_DB_to_table(DBname = 'intake', keyword = 'labnosh')
lnsshop.shop_DB_to_table('intake', 'labnosh')
lnsblog.blog_DB_to_table('intake', 'labnosh')
pckinsta.pickle_to_table('Data/pck_list.txt')
pckshop.shop_DB_to_table('intake', 'pck')
pckblog.blog_DB_to_table('intake', 'pck')
kgcinsta.pickle_to_table('Data/kgc_list.txt')
kgcshop.shop_DB_to_table('intake', 'kgc')
kgcblog.blog_DB_to_table('intake', 'kgc')



                   user_id          created_at  \
0        invincible_skhyun 2018-01-24 11:11:08   
1                 jiyu_lee 2018-01-24 08:37:57   
2               dionycchus 2018-01-24 07:43:53   
3        invincible_skhyun 2018-01-23 12:55:54   
4               diet.oneul 2018-01-23 12:09:54   
5        styleshare_beauty 2018-01-23 08:13:52   
6              ddong940423 2018-01-23 07:59:04   
7               ssujjeong_ 2018-01-23 07:32:40   
8             dameulstudio 2018-01-23 05:54:14   
9                catharen_ 2018-01-23 00:22:17   
10              lora.study 2018-07-03 23:57:47   
11                1122_asm 2018-07-02 01:08:57   
12             makemysugar 2018-07-05 13:13:05   
13           oasis_jaeyoon 2018-06-28 01:59:52   
14                khjhmom_ 2018-07-04 11:01:04   
15                leeminwk 2018-06-12 11:05:21   
16                   vveun 2018-06-13 13:26:21   
17            _si1ver_star 2018-01-14 12:30:54   
18          kyu_ri_the_one 2018-04-21 12:56:13   


                 user_id          created_at  \
0            oli_hyunwoo 2017-01-10 14:15:07   
1                hjw2151 2017-01-10 13:09:17   
2               yoonazzo 2017-01-10 09:11:26   
3               yoonazzo 2017-01-10 09:07:47   
4                sue_960 2017-01-10 09:06:26   
5         adorable_jeong 2017-01-10 07:33:42   
6                 sjtvgo 2017-01-10 07:27:48   
7                 ha.ray 2017-01-10 07:05:52   
8             kimsweet96 2017-01-10 05:32:05   
9            with_homeal 2017-01-10 05:31:04   
10               ajs0921 2017-01-10 04:47:21   
11     atemshop.official 2017-01-10 01:17:35   
12              zizi_44d 2017-01-10 00:08:10   
13              cha_ella 2017-01-09 22:24:44   
14             jungg1_su 2017-01-09 15:13:43   
15       allbymyfoodie_t 2017-01-09 13:16:31   
16             ssun_unni 2017-01-09 12:49:40   
17              kdg16096 2017-01-09 10:50:10   
18              ssssolra 2017-01-09 10:31:01   
19                 ppo.d 2017-01-09 09:0

In [6]:
itkinsta.del_list.extend(['독립운동', '현정'])
itkshop.del_list.extend(['독립운동', '현정'])
itkblog.del_list.extend(['독립운동', '현정'])


pckinsta.del_list.extend(['서주현','서현','계피','차돌박이','삼계탕','닭꼬치','볶음밥'])
pckshop.del_list.extend(['노브랜드', '라자냐', '티라미수', '하노이', '빈대떡', '서주현','서현','계피','차돌박이','삼계탕','닭꼬치','볶음밥'])
pckblog.del_list.extend(['노브랜드', '라자냐', '티라미수', '하노이', '빈대떡', '서주현','서현','계피','차돌박이','삼계탕','닭꼬치','볶음밥'])

kgcinsta.del_list.extend(['에브리타임','정관장','홍삼정','정해인','정관','장홍','조정석','디페','전광렬','홍삼스틱','한국인삼공사','홍이장군','박은빈'])
kgcshop.del_list.extend(['에브리타임','정관장','홍삼정','정해인','정관','장홍','조정석','디페','전광렬','홍삼스틱','한국인삼공사','홍이장군','박은빈'])
kgcblog.del_list.extend(['에브리타임','정관장','홍삼정','정해인','정관','장홍','조정석','디페','전광렬','홍삼스틱','한국인삼공사','홍이장군','박은빈'])




###lnsinsta.del_list.extend([''])###

In [ ]:
lnsinsta.df

In [7]:
itkinsta.df = itkinsta.word_check('or', ['자동차', '흡기', '배기','도어락'], itkinsta.df, 'hashtags', False)
itkinsta.df = itkinsta.word_check('or', ['intakefoods', 'dameulstudio', '_.ddo2', '__scarlett.k', '0.8l_korea', 'jiseung86', 'untactmarket'], itkinsta.df, 'user_id', False)
itkinsta.df = itkinsta.word_check('or', ['자동차', '흡기', '배기','도어락'], itkinsta.df, 'main_text', False)
lnsinsta.df = lnsinsta.word_check('or', ['리그램','리포스트','리그'], lnsinsta.df, 'main_text', False)
lnsinsta.df = lnsinsta.word_check('or', ['리그램','리포스트','리그','매일다른패션_매일새로운랩노쉬'], lnsinsta.df, 'hashtags', False)


In [29]:
itkinsta.load_dictionary()
itkshop.load_dictionary()
itkblog.load_dictionary()

lnsinsta.load_dictionary()
lnsshop.load_dictionary()
lnsblog.load_dictionary()

pckinsta.load_dictionary()
pckshop.load_dictionary()
pckblog.load_dictionary()

kgcinsta.load_dictionary()
kgcshop.load_dictionary()
kgcblog.load_dictionary()

conver

temp = df_new_intake.loc[df_new['main_text'].str.contains('맛') & df_new['main_text'].str.contains('모닝죽') & df_new['main_text'].str.contains('있')]['']
temp.describe()

In [8]:
itkinsta.morph_pos(itkinsta.df['main_text'])

itkshop.morph_pos(itkshop.df['main_text'])

itkblog.morph_pos(itkblog.df['main_text'])

lnsinsta.morph_pos(lnsinsta.df['main_text'])

lnsshop.morph_pos(lnsshop.df['main_text'])

lnsblog.morph_pos(lnsblog.df['main_text'])

pckinsta.morph_pos(pckinsta.df['main_text'])

pckshop.morph_pos(pckshop.df['main_text'])

pckblog.morph_pos(pckblog.df['main_text'])

kgcinsta.morph_pos(kgcinsta.df['main_text'])

kgcshop.morph_pos(kgcshop.df['main_text'])

kgcblog.morph_pos(kgcblog.df['main_text'])


[['월드_NNG',
  '디제이_NNG',
  '페스티벌_NNG',
  '후기_NNG',
  '월_NNG',
  '디페_NNG',
  '다녀왔_VV+EP',
  '어요_EF',
  '정관장_NNG',
  '에브리타임_NNG',
  '먹_VV',
  '고_EC',
  '버_NNP',
  '닝_NNP',
  '내_NP+JKG',
  '생애_NNG',
  '첫_MM',
  '월드_NNG',
  '디제이_NNG',
  '페스티벌_NNG',
  '다른_MM',
  '축제_NNG',
  '는_JX',
  '많이_MAG',
  '가_VV+EC',
  '봤_VX+EP',
  '는데_EC',
  '월_NNG',
  '디페_NNG',
  '랑_JKB',
  '은_JX',
  '이상_NNG',
  '하_XSV',
  '게_EC',
  '인연_NNG',
  '이_JKS',
  '없_VA',
  '더라고요_EC',
  'ㅠㅠ_UNKNOWN',
  '올해_NNG',
  '는_JX',
  '드디어_MAG',
  '다녀왔_VV+EP',
  '어요_EF',
  '저_NP',
  '는_JX',
  '일요_NNG',
  '일_NR',
  '원_NNBC',
  '데이_NNG',
  '권_XSN',
  '으로_JKB',
  '다녀왔_VV+EP',
  '는데_EC',
  '토요일_NNG',
  '은_JX',
  '많이_MAG',
  '더웠_VA+EP',
  '다고_EC',
  '하_VV',
  '더라고요_EC',
  '일요일_NNG',
  '선택_NNG',
  '한_XSA+ETM',
  '거_NNB',
  '아주_MAG',
  '칭찬_NNG',
  '해_XSV+EC',
  '잠실_NNP',
  '종합_NNG',
  '운동장_NNG',
  '서울특별시_NNP',
  '송파구_NNP',
  '올림픽로_NNP',
  '서울_NNP',
  '종합_NNG',
  '운동장_NNG',
  '지도_NNG',
  '보_VV',
  '기_ETN',
  '우리_NP',
  '집_NNG',
  '에서_JKB',
  '

In [ ]:
kgcshop.df['morph_list']

In [9]:
itkinsta.pos_extractor(itkinsta.df['morph_list'], 'set', 'theme')
itkshop.pos_extractor(itkshop.df['morph_list'],'list', 'theme')
itkblog.pos_extractor(itkblog.df['morph_list'],'list', 'theme')

lnsinsta.pos_extractor(lnsinsta.df['morph_list'],'set', 'theme')
lnsshop.pos_extractor(lnsshop.df['morph_list'],'list', 'theme')
lnsblog.pos_extractor(lnsblog.df['morph_list'],'list', 'theme')

pckinsta.pos_extractor(pckinsta.df['morph_list'],'set', 'theme')
pckshop.pos_extractor(pckshop.df['morph_list'],'list', 'theme')
pckblog.pos_extractor(pckblog.df['morph_list'],'list', 'theme')

kgcinsta.pos_extractor(kgcinsta.df['morph_list'],'set', 'theme')
kgcshop.pos_extractor(kgcshop.df['morph_list'],'list', 'theme')
kgcblog.pos_extractor(kgcblog.df['morph_list'],'list', 'theme')

In [10]:
itkinsta.df

,user_id,created_at,main_text,hashtags,comments,likes,current_url,flags,morph_list,nav_list,noun_list,adj_list,verb_list
0,invincible_skhyun,2018-01-24 11:11:08,invincible_skhyun#식단일기_180124\n.\n아침:인테이크 소이밀크...,식단일기_180124/인테이크/인테이크소이밀크/아임웰/아임웰굿밸런스라이스/아임닭/아...,,11,https://www.instagram.com/p/BeVH5d3luGZ/?hl=ko...,0,"[식단_NNG, 일기_NNG, 아침_NNG, 인테이크_NNG, 소이_NNP, 밀크_...","[식단, 일기, 아침, 소이, 밀크, 블랙, 점심, 아임, 굿, 밸런스, 라이스, ...","[식단, 일기, 아침, 소이, 밀크, 블랙, 점심, 아임, 굿, 밸런스, 라이스, ...",[],[]
1,jiyu_lee,2018-01-24 08:37:57,jiyu_lee-\n요즘 먹고 있는 유산균.\n-\n인테이크에서 판매하는걸 보고 주...,인테이크/intake/휴먼바이오틱스/유산균/포장이예쁘니/효과도좋을거같고,,19,https://www.instagram.com/p/BeU2Xe0BWpD/?hl=ko...,0,"[요즘_NNG, 먹_VV, 고_EC, 있_VX, 는_ETM, 유산균_NNG, 인테이...","[요즘, 유산균, 판매, 주문, 괜찮, 보통, 알약, 개별, 포장, 가루, 물, 흔...","[요즘, 유산균, 판매, 주문, 보통, 알약, 개별, 포장, 가루, 물, 타입, 마...","[흔들, 마시, 따라서, 마셔야, 바뀌, 모르]","[괜찮, 귀찮, 예쁘]"
2,dionycchus,2018-01-24 07:43:53,dionycchus#인테이크\n올해엔 건강해져볼까 합니다ㅎㅎ\n이러고 또 술처먹겠지...,인테이크,,11,https://www.instagram.com/p/BeUwLe9lTrg/?hl=ko...,0,"[인테이크_NNG, 올해_NNG, 엔_JKB+JX, 건강_NNG, 해져_XSA+EC...","[올해, 건강, 이러, 처먹]","[올해, 건강]","[이러, 처먹]",[]
3,invincible_skhyun,2018-01-23 12:55:54,invincible_skhyun#식단일기_180123\n\n다시 마음잡고 시작해보는...,식단일기_180123/인테이크/인테이크소이밀크/아임웰/아임웰라이트밀/아임닭/아임닭스...,,,https://www.instagram.com/p/BeSvF-eF6wk/?hl=ko...,0,"[식단_NNG, 일기_NNG, 다시_MAG, 마음잡_VV, 고_EC, 시작_NNG,...","[식단, 일기, 마음잡, 시작, 아침, 두유, 점심, 아임, 라이트, 카레, 큐브,...","[식단, 일기, 시작, 아침, 두유, 점심, 아임, 라이트, 카레, 큐브, 간식, ...",[마음잡],[재미있]
4,diet.oneul,2018-01-23 12:09:54,diet.oneul#오늘의식단\n#저녁\n인테이크 모닝죽 단팥.\n냉동야채+닭쌤닭가...,오늘의식단/저녁/식단/식단일기/인테이크/모닝죽,,15,https://www.instagram.com/p/BeSp1AIl7tT/?hl=ko...,0,"[오늘_NNG, 의_JKG, 식단_NNG, 저녁_NNG, 인테이크_NNG, 모닝죽단...","[오늘, 식단, 저녁, 모닝죽, 냉동야채, 가슴살, 소세지, 일기]","[오늘, 식단, 저녁, 모닝죽, 냉동야채, 가슴살, 소세지, 일기]",[],[]
5,styleshare_beauty,2018-01-23 08:13:52,styleshare_beauty[#뷰티실험실 : 다이어트 핫템 4🔥]\n굶는 다이어...,뷰티실험실/인테이크/스타일쉐어/스쉐스토어/다이어트/대용식/식단조절,,,https://www.instagram.com/p/BeSOvOigMHJ/?hl=ko...,0,"[뷰티_NNG, 실험실_NNG, 다이어트_NNG, 핫_NNG, 템_NNG, 굶_VV...","[뷰티, 실험실, 다이어트, 건강, 도와, 가지, 이젠, 대용식, 스토어, 최대, ...","[뷰티, 실험실, 다이어트, 건강, 가지, 이젠, 대용식, 스토어, 최대, 할인, ...",[도와],[]
6,ddong940423,2018-01-23 07:59:04,ddong940423#intake #인테이크 #밀스3 #이벤트\n1.23 무료체험 ...,intake/인테이크/밀스3/이벤트/stopsilver425/hello.oioi/d...,,19,https://www.instagram.com/p/BeSNHz-jPT5/?hl=ko...,0,"[인테이크_NNG, 밀스_NNP, 이벤트_NNG, 무료_NNG, 체험_NNG, 이벤...","[이벤트, 무료, 체험, 아침, 필요, 없]","[이벤트, 무료, 체험, 아침, 필요]",[],[없]
7,ssujjeong_,2018-01-23 07:32:40,ssujjeong_군것질을 절대 하지않기위해 다이어트쿠키를 샀당\n다욧하는사람들 많...,몸스터즈/다이어트쿠키/다이어트시작한닷/운동하는여자/운동하는남자/운동/헬스타그램/비키...,ssujjeong_.\n.\n.\n#운동하는여자#운동하는남자#운동#헬스타그램#비키니...,132,https://www.instagram.com/p/BeSKGh5jRet/?hl=ko...,0,"[군것질_NNG, 을_JKO, 절대_MAG, 하_VV, 지_EC, 않_VX, 기_E...","[군것질, 위해, 다이어트, 쿠키, 사람, 박스, 리아, 챙겨, 보내, 마시, 몸,...","[군것질, 다이어트, 쿠키, 사람, 박스, 리아, 마시, 몸, 스터, 시작]","[위해, 챙겨, 보내]",[]
9,catharen_,2018-01-23 00:22:17,catharen_:\n생명 유지 수단이라고나 할까\n\n#일상 #작업일기 #인테이크...,일상/작업일기/인테이크/두유/모닝스타그램/dailylife/dailywork/soy...,harumulgorae잘보고 가용!/tangbong_91잘보고 가요 소통해요 😎,48,https://www.instagram.com/p/BeRY2S6FDRW/?hl=ko...,0,"[생명_NNG, 유지_NNG, 수단_NNG, 이_VCP, 라고_EC, 나_JX, 할...","[생명, 유지, 수단, 할까, 일상, 작업, 일기, 두유, 모닝, 스타그램]","[생명, 유지, 수단, 일상, 작업, 일기, 두유, 모닝, 스타그램]",[할까],[]
10,lora.study,2018-07-03 23:57:47,lora.study#아침식사#죽#모닝죽#단호박#인테이크\n.\n몇년째 먹는 모닝죽\...,아침식사/죽/모닝죽/단호박/인테이크/맛스타그램/lfl/foodporn/먹스타그램/식...,lora.study#맛스타그램#lfl#foodporn#먹스타그램#식사대용#다이어트#...,163,https://www.instagram.com/p/Bkye0hsFHJB/?hl=ko...,0,"[아침_NNG, 식사_NNG, 죽_NNG, 모닝죽단호박_NNG, 인테이크_NNG, ...","[아침, 식사, 바쁘, 입맛, 없, 맛, 종류, 있, 맛있, 고르, 모닝죽, 나온]","[아침, 식사, 입맛, 맛, 종류, 모닝죽]","[고르, 나온]","[바쁘, 없, 있, 맛있]"


itkinsta.nav_merged = itkinsta.merge_list(itkinsta.df['nav_list'])
itkshop.nav_merged = itkshop.merge_list(itkshop.df['nav_list'])
itkblog.nav_merged = itkblog.merge_list(itkblog.df['nav_list'])

lnsinsta.nav_merged = lnsinsta.merge_list(lnsinsta.df['nav_list'])
lnsshop.nav_merged = lnsshop.merge_list(lnsshop.df['nav_list'])
lnsblog.nav_merged = lnsblog.merge_list(lnsblog.df['nav_list'])

pckinsta.nav_merged = pckinsta.merge_list(pckinsta.df['nav_list'])
pckshop.nav_merged = pckshop.merge_list(pckshop.df['nav_list'])
pckblog.nav_merged = pckblog.merge_list(pckblog.df['nav_list'])

kgcinsta.nav_merged = kgcinsta.merge_list(kgcinsta.df['nav_list'])
kgcshop.nav_merged = kgcshop.merge_list(kgcshop.df['nav_list'])
kgcblog.nav_merged = kgcblog.merge_list(kgcblog.df['nav_list'])


In [ ]:
itkinsta.nav_merged

In [10]:
itkinsta.nav_joined = itkinsta.join_list(itkinsta.df['nav_list'])
itkshop.nav_joined = itkshop.join_list(itkshop.df['nav_list'])
itkblog.nav_joined = itkblog.join_list(itkblog.df['nav_list'])

lnsinsta.nav_joined = lnsinsta.join_list(lnsinsta.df['nav_list'])
lnsshop.nav_joined = lnsshop.join_list(lnsshop.df['nav_list'])
lnsblog.nav_joined = lnsblog.join_list(lnsblog.df['nav_list'])

pckinsta.nav_joined = pckinsta.join_list(pckinsta.df['nav_list'])
pckshop.nav_joined = pckshop.join_list(pckshop.df['nav_list'])
pckblog.nav_joined = pckblog.join_list(pckblog.df['nav_list'])

kgcinsta.nav_joined = kgcinsta.join_list(kgcinsta.df['nav_list'])
kgcshop.nav_joined = kgcshop.join_list(kgcshop.df['nav_list'])
kgcblog.nav_joined = kgcblog.join_list(kgcblog.df['nav_list'])

In [33]:
lnsinsta.nav_joined

['교무실 책상 방학 생활 기록부 작성 아침 식사 가볍 완벽 여자 마음 사로잡 마셔 배부름 미래형 올리브영 겨울 맛있 그램 수국 꽃다발 대일 외고 토마토 베이직 그레인 밀크 자색고구마 간편식 다이어트 건강식 먹스타그램 대용식',
 '절대 다이어트식 다이어터 칼로리 지방 포화 나트륨 콜레스테롤 밥 싫 포만감 원할 마시 찬물 풀려요',
 '비교',
 '',
 '성공 맛있 먹스타그램',
 '밥 시간 없 요걸로 그레인 보건 학원 의원',
 '아침밥 건강 챙기 고프 살찔 일상',
 '출근 위해 구입 시작 미래형 식사 베이직 편함 맛있 파랑 먹스타그램 희망',
 '고객 후기 감사 건강 챙기 남자 호밀 슬림 식단 친구 만나 회식 있 사람 기분 그때 대비 평소 식단기록 습관 생겼 도중 밥 들어가 다이어트 도움 조절 해야 트루 라이프 주일 신청 아침 배달 안심 가지찜 소스 특이 맛 전자레인지 돌리 오늘 후라이팬 맛있 미래형 식사 후식 물 단백질 건강보조식품 아마조니아 빡세 예정 도시락 저녁 스타그램 먹스타그램 소통 댓글',
 '광고 봐서 초코 생각 맛있 초딩 입맛 맛 식사대용 다이어트 괜찮 올리브영',
 '동물 실험 대신 직원 섭취 식품 지속 퀄리티 체크 개선 신경 답니다 댓글 리뷰 말씀 피드백 개발 전달 발전 반영 이번 단맛 맛 줄이 부정 평가 부분 이슈 미래형 식사 시음 장려 맛있 다음 고객',
 '아침 쾌변 다이어터 다이어트 소통 일기 식단 팔로우 먹스타그램 초코',
 '토마토 때리 나오 버스 놓쳤 힘내 망할 이아이 맛 평하 주스 유청 단백질 없 거슬림 자색고구마 거야 오늘 아침 바나나 주원 공복 운동 완료',
 '근무 아침 출근길 물 이킹 쉐이 맛 종류 다양 달 초코 우유 기분 월요일 지역 춥다 시려 올리브영 영양가 식사대용 포만감 시간 다음 자색고구마',
 '베이직',
 '오늘 저녁 베이직 맛있 있 미래형 식사 은근 배부르 야식 땡기 다이어트 소통 인친',
 '건강 챙기 남자 호밀 슬림 식단 친구 만나 회식 있 사람 기분 그때 대비 평소 식단기록 습관 생겼 도중 밥 

In [11]:
itk_joined = itkinsta.nav_joined + itkshop.nav_joined + itkblog.nav_joined
lns_joined = lnsinsta.nav_joined + lnsshop.nav_joined + lnsblog.nav_joined
pck_joined = pckinsta.nav_joined + pckshop.nav_joined + pckblog.nav_joined
kgc_joined = kgcinsta.nav_joined + kgcshop.nav_joined + kgcblog.nav_joined

In [12]:
intake= Social_analysis('인테이크')
labnosh =Social_analysis('랩노쉬')


## 1. intake insta- list - sys

In [ ]:
itkinsta.LDA = SB_LDA()

In [ ]:
itkinsta.LDA.make_lda(itkinsta.nav_joined, ntopic=2, learning_method='batch', max_iter=25, random_state=0, n_words=20)

In [ ]:
itkinsta.LDA.make_lda(itkinsta.nav_joined, ntopic=3, learning_method='batch', max_iter=25, random_state=0, n_words=20)

In [148]:
itkinsta.LDA.make_lda(itkinsta.nav_joined, ntopic=4, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
먹스타그램         젤리            향신료           운동            
모닝죽           홍삼            보내            감사            
영양            힘내            세상            견과            
건강스타그램        홍삼젤리스틱        조미료           프리미엄          
봉지            추천            오늘            부모            
아이            하루            요리            준비            
타임            비타민           집들이           저녁            
운동            피로            다이어터          슈퍼            
오늘            식품            모닝죽           다이어터          
식품            챙겨            다이어트식단        고민            
맥주            명절            식단            다이어트식단        
안주            스틱            소통            샐러드           
먹방            데일리           데일리           오늘            
한끼            감사            기록            데일리           
식사            소통            명절            고급            
밀스            운동            고마워

In [149]:
itkinsta.LDA.make_lda(itkinsta.nav_joined, ntopic=5, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
먹스타그램         홍삼            보내            프리미엄          향신료           
운동            젤리            모닝죽           견과            요리            
식단            힘내            향신료           고급            집들이           
오늘            홍삼젤리스틱        챙겨            데일리           오늘            
모닝죽           하루            조미료           준비            운동            
밀스            챙겨            고마워           부모            견과            
식사            추천            감사            고민            세상            
타임            명절            데일리           슈퍼            신혼            
생각            식품            행복            챙겨            식단            
조절            감사            다이어터          감사            만들            
사과            피로            주문            푸드            사진            
소통            소통            취향            운동            저녁            
식품    

In [150]:
itkinsta.LDA.make_lda(itkinsta.nav_joined, ntopic=6, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
생각            감사            보내            데일리           자랑            
타임            챙겨            향신료           언니            프리미엄          
먹스타그램         비타민           모닝죽           생일            견과            
아이            친구            요리            오늘            캐슈            
영양            이번            감사            감사            푸드            
술안주           아이            조미료           샐러드           운동            
맥주            오늘            집들이           고마워           아몬드           
안주            추천            챙겨            다이어트식단        명절            
운동            하루            세상            고민            가지            
밀스            준비            모닝죽단호박        모닝죽           오늘            
어린이           안주            기분            위해            부모            
건강스타그램        보내            택배            카페            식사대용          
하루견과  

## 2. labnosh insta

In [19]:
labnosh.LDA = SB_LDA()

In [20]:
labnosh.LDA.make_lda(lnsinsta.nav_joined, ntopic=2, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       
--------      --------      
추석            식단            
세트            오늘            
올리브           한끼            
힐링            다이어트식단        
영행            사진            
시간            다이어터          
대전            운동            
리듬            랩노쉬플랫         
카밀            감사            
챕스틱           일기            
영추            미식당           
케어            이벤트           
기프트           호박즙           
이크            간식            
감사            그램            
친구            플랫바           
챙겨            저녁            
고마워           책상위작은밭        
사랑            단백질           
미래형           쉐이크           




In [21]:
labnosh.LDA.make_lda(lnsinsta.nav_joined, ntopic=3, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       topic 2       
--------      --------      --------      
추석            호박즙           식단            
올리브           책상위작은밭        감사            
세트            사진            오늘            
힐링            랩노쉬플랫         한끼            
영행            오늘            쉐이크           
대전            아로니아즙         그램            
시간            위한            생일            
챕스틱           패키지           운동            
리듬            미식당           다이어트식단        
카밀            이벤트           고마워           
영추            다이어터          점심            
케어            감동            간식            
기프트           생각            조절            
이크            식단            저녁            
친구            다이어트식단        식사대용          
챙겨            감사            푸드쉐이크         
맞팔            일기            출근            
미래형           모르            다이어터          
그램            엄마            사랑            
감사            행복            마음            




In [22]:
labnosh.LDA.make_lda(lnsinsta.nav_joined, ntopic=4, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
올리브           호박즙           감사            사랑            
추석            책상위작은밭        식단            감사            
세트            식단            쉐이크           한끼            
영행            랩노쉬플랫         한끼            고마워           
힐링            패키지           그램            생각            
대전            위한            출근            쇼콜라           
챕스틱           다이어트식단        생일            이벤트           
리듬            아로니아즙         다이어트식단        센스            
카밀            일기            운동            마시            
영추            오늘            식사대용          사진            
시간            사진            다이어터          오늘            
케어            친구            간식            카페            
기프트           다이어터          저녁            와플            
이크            운동            오늘            미래            
친구            단백질           조절            회사            
챙겨            챙겨            요거트

In [23]:
labnosh.LDA.make_lda(lnsinsta.nav_joined, ntopic=5, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
추석            호박즙           그램            감사            오늘            
올리브           식단            감사            한끼            미래형           
세트            책상위작은밭        쉐이크           생각            저녁            
영행            패키지           한끼            사랑            식단            
힐링            일기            생일            마시            다이어트식단        
대전            랩노쉬플랫         식사대용          호박즙           센스            
리듬            위한            출근            하루            맞팔            
챕스틱           이벤트           운동            고마워           다이어터          
카밀            사진            식단            감동            랩노쉬플랫         
영추            아로니아즙         동생            커피            운동            
시간            하루            챙겨            사진            그린씨리얼         
기프트           이번            인증샷           이벤트           쉐이크           
케어    

## 피코크

In [47]:
pck.LDA = SB_LDA()

In [ ]:
pck.LDA.make_lda(pck_joined, ntopic=2, learning_method='batch', max_iter=25, random_state=0, n_words=20)

## 정관장

In [ ]:
kgc.LDA = SB_LDA()

In [ ]:
kgc.LDA.make_lda(kgc_joined, ntopic=2, learning_method='batch', max_iter=25, random_state=0, n_words=20)

In [13]:
intake.LDA = SB_LDA()

In [14]:
intake.LDA.make_lda(itk_joined, ntopic=2, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       
--------      --------      
다이어트          모닝죽           
밀스            괜찮            
식사대용          배송            
식사            구매            
우유            주문            
식단            가격            
간식            선물            
건강            생각            
다이어터          식사대용          
운동            제품            
스타그램          고구마           
저녁            만족            
그램            빠르            
식단기록          감사            
오늘            귀리            
칼로리           박스            
일상            포장            
두유            건강            
점심            구입            
마시            챙겨            




In [15]:
intake.LDA.make_lda(itk_joined, ntopic=3, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       topic 2       
--------      --------      --------      
다이어트          배송            모닝죽           
식단            괜찮            식사대용          
다이어터          선물            밀스            
스타그램          구매            식사            
운동            가격            우유            
그램            감사            다이어트          
간식            만족            생각            
식단기록          견과류           칼로리           
일상            포장            고구마           
오늘            제품            제품            
저녁            곤약젤리          주문            
건강            건강            구매            
점심            대비            마시            
모닝죽           간식            느낌            
소통            젤리            빠르            
일기            주문            귀리            
먹스타그램         맛나            건강            
밀스            상품            두유            
조절            박스            정도            
데일리           빠른            포만감           




In [16]:
intake.LDA.make_lda(itk_joined, ntopic=4, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
다이어트          배송            모닝죽           간식            
스타그램          괜찮            밀스            다이어트          
다이어터          구매            식사대용          칼로리           
식단            선물            식사            곤약젤리          
그램            가격            우유            건강            
식단기록          만족            고구마           생각            
운동            감사            제품            과자            
일상            주문            다이어트          모닝            
오늘            재구매           빠르            수프            
저녁            포장            구매            젤리            
소통            제품            생각            영양            
모닝죽           대비            칼로리           쿠키            
일기            견과류           귀리            가지            
먹스타그램         맛나            주문            조절            
점심            상품            마시            봉지            
건강            구입            두유 

In [17]:
intake.LDA.make_lda(itk_joined, ntopic=5, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
다이어트          선물            모닝죽           간식            배송            
다이어터          구매            밀스            다이어트          괜찮            
식단            견과류           식사대용          칼로리           가격            
스타그램          건강            식사            곤약젤리          빠르            
식단기록          맛나            우유            생각            감사            
그램            포장            고구마           과자            만족            
일상            제품            제품            쿠키            재구매           
오늘            비타민           구매            젤리            주문            
저녁            챙겨            다이어트          건강            대비            
모닝죽           젤리            귀리            조절            구매            
운동            마음            생각            봉지            빠른            
점심            가지            칼로리           추천            상품            
소통    

In [24]:
intake.LDA.make_lda(itk_joined, ntopic=6, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
다이어트          선물            모닝죽           간식            배송            
다이어터          견과류           밀스            다이어트          괜찮            
식단            건강            식사대용          칼로리           가격            
식단기록          구매            식사            곤약젤리          구매            
그램            포장            우유            생각            만족            
스타그램          맛나            고구마           과자            빠르            
일상            비타민           제품            젤리            감사            
오늘            챙겨            다이어트          쿠키            주문            
저녁            푸드            귀리            조절            재구매           
모닝죽           제품            마시            건강            대비            
점심            젤리            칼로리           봉지            상품            
일기            하루            구매            인테이크모닝그래놀라    빠른            
먹스타그램 

In [19]:
labnosh.LDA = SB_LDA()

In [21]:
labnosh.LDA.make_lda(lns_joined, ntopic=2, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       
--------      --------      
다이어터          쉐이크           
식단기록          푸드쉐이크         
건강            마시            
일기            시간            
간식            자색고구마         
운동            생각            
책상위작은밭        칼로리           
그램            포만감           
점심            일상            
뮤즐리           흔들            
두유            가지            
스타그램          초코            
관리            제품            
일상            우유            
하루            건강            
푸드            스타그램          
랩노쉬플랫         후기            
단백질           요거트           
시작            느낌            
소통            먹스타그램         




In [22]:
labnosh.LDA.make_lda(lns_joined, ntopic=3, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       topic 2       
--------      --------      --------      
다이어터          쉐이크           일상            
식단기록          푸드쉐이크         스타그램          
일기            마시            건강            
간식            포만감           먹스타그램         
점심            시간            책상위작은밭        
두유            생각            소통            
뮤즐리           흔들            데일리           
운동            가지            하루            
단백질           자색고구마         푸드            
랩노쉬플랫         칼로리           시작            
그램            제품            관리            
조절            초코            이벤트           
모닝죽           우유            선물            
미식당           후기            그램            
플랫바           요거트           스타일           
건강            미숫가루          새로운           
기록            느낌            리쉬            
프로            가루            영양            
관리            건강            제품            
아몬드           구매            완벽            




In [23]:
labnosh.LDA.make_lda(lns_joined, ntopic=4, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
칼로리           다이어터          책상위작은밭        일상            
생각            식단기록          건강            스타그램          
흔들            일기            모닝죽           먹스타그램         
쉐이크           푸드쉐이크         푸드            소통            
우유            점심            이벤트           데일리           
가지            운동            하루            자색고구마         
초코            그램            스타일           마시            
제품            조절            시작            건강            
시간            랩노쉬플랫         새로운           시간            
마시            간식            리쉬            점심            
건강            단백질           제품            그램            
포만감           관리            선물            쉐이크           
느낌            기록            영양            푸드쉐이크         
단백질           뮤즐리           완벽            하루            
정도            플랫바           패키지           출근            
요거트           소통            관리 

In [20]:
labnosh.LDA.make_lda(lns_joined, ntopic=5, learning_method='batch', max_iter=25, random_state=0, n_words=20)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
두유            쉐이크           식단기록          일상            책상위작은밭        
단백질           푸드쉐이크         다이어터          스타그램          건강            
뮤즐리           흔들            일기            소통            모닝죽           
간식            마시            운동            먹스타그램         하루            
건강            가지            점심            데일리           푸드            
초코            포만감           그램            자색고구마         시작            
랩노쉬플랫         시간            조절            시간            스타일           
우유            요거트           랩노쉬플랫         그램            리쉬            
칼로리           가루            관리            건강            이벤트           
생각            자색고구마         간식            점심            제품            
느낌            칼로리           기록            쉐이크           새로운           
현미            생각            스타그램          마시            완벽            
식감    

In [ ]:
labnosh.LDA.make_lda(lns_joined, ntopic=6, learning_method='batch', max_iter=25, random_state=0, n_words=20)

In [51]:
'독립운동' in itkinsta.nav_merged

False

In [52]:
'인테이크검은콩' in itkinsta.nav_merged

True

In [53]:
pd.options.display.max_colwidth = 2000

In [54]:
itk_merged = itkinsta.nav_merged + itkshop.nav_merged + itkblog.nav_merged
lns_merged = lnsinsta.nav_merged + lnsshop.nav_merged + lnsblog.nav_merged
pck_merged = pckinsta.nav_merged + pckshop.nav_merged + pckblog.nav_merged
kgc_merged = kgcinsta.nav_merged + kgcshop.nav_merged + kgcblog.nav_merged

In [55]:
tfidf = SB_Tfidf([itk_merged, lns_merged])
tfidf.get_tfidf()
tfidf_of_all = tfidf.tfidf_hangul

for i in tfidf_of_all:
    pprint(i[:40])
    print()

[('인테이크단호박죽', 0.5617447591606419),
 ('인테이크검은콩', 0.420043378471471),
 ('다이어트칩', 0.2361689678152849),
 ('홍삼젤리스틱', 0.2108651498350758),
 ('우유죽', 0.13832753829180972),
 ('모닝죽우유', 0.1315798534970873),
 ('프립', 0.12483216870236487),
 ('컴포트', 0.12314524750368427),
 ('슈퍼스무디', 0.12145832630500367),
 ('밀스드링크딸기', 0.11639756270896184),
 ('버터치킨커리', 0.11639756270896184),
 ('홍삼구미', 0.10121527192083639),
 ('조미료', 0.09615450832479457),
 ('퍼드', 0.09615450832479457),
 ('아해', 0.09278066592743335),
 ('카페다이어트', 0.08940682353007214),
 ('귀리죽', 0.08434605993403033),
 ('모닝바게트', 0.08434605993403033),
 ('소백산', 0.08265913873534972),
 ('시스', 0.08265913873534972),
 ('정모', 0.08097221753666911),
 ('모닝죽검은콩', 0.0775983751393079),
 ('밀스라이트카페라떼', 0.07591145394062729),
 ('아카시아', 0.07422453274194668),
 ('스크루', 0.07253761154326607),
 ('곤약현미밥', 0.07085069034458547),
 ('홍경', 0.07085069034458547),
 ('산행', 0.06916376914590486),
 ('선스', 0.06072916315250183),
 ('스윗그레인', 0.06072916315250183),
 ('지미', 0.06072916315250183),
 ('품질', 0.

In [45]:
for x in lnsinsta.df['nav_list']:
    if '리쉬' in x:
        print (x)

['굿모닝', '월요일', '월요병', '도져서', '늦잠', '덕분', '지각', '면했', '신호등', '사진', '여유', '미래형', '식사', '건강', '한끼', '완벽', '한끼', '하루', '영양', '식사', '스타일', '리쉬', '푸드']
['디자이너', '제안', '자색고구마', '미래형', '식사', '자색고구마', '건강', '한끼', '완벽', '한끼', '하루', '영양', '식사', '스타일', '리쉬', '푸드', '먹스타그램']
['디자이너', '이러', '자색고구마', '무지개떡', '미래형', '식사', '건강', '한끼', '완벽', '한끼', '하루', '영양', '식사', '스타일', '리쉬', '푸드', '먹스타그램']
['민트', '배경', '상세', '후기', '건강', '챙기', '남자', '오픈', '바빠', '주문', '아침', '가방', '속', '물', '한끼', '해결', '편해요', '물', '맛있', '단순', '선식', '텐데', '탄단', '영양소', '미네랄', '비타민', '미래형', '식사', '이번', '지난번', '맛', '입맛', '자색고구마', '그래놀라요거트', '마이', '키트', '주문', '추가', '부스터', '구매', '피시', '콜라겐', '몸', '흡수', '피부', '스킨', '부스터', '스트레스', '완화', '스트레스릴리프', '부스터', '구매', '챙겨', '하루', '건강', '아침', '아침', '점심', '저녁때', '폭식', '건강', '악화', '있', '아침', '공복', '출근길', '부스터', '아침', '고파요', '폭식', '습관', '바꾸', '나중', '쉬워', '아침', '챙겨', '드세요', '미래형', '식사', '건강', '한끼', '완벽', '한끼', '하루', '영양', '식사', '다이어트', '스타일', '리쉬', '푸드']
['새해', '다이어트', '동지', '모두', '화이팅', '소망', '후기', '늘어난', '

## csv에 안쓰고 이런식으로도 추가 가능. 하지만 csv에 자동저장되진 않는다.

단어_태그_T/F(받침여부)_원단어의발음

## m이 False인 경우에는 진짜 맛있다는 것.
## -> '없' 이라는 음절이 '있'이라는 음절 이후 4음절 내에 없다는 뜻이다.
## m이 True인 경우에는 애매한 상황이다. '없' 이라는 음절이 4음절 내에 존재한다는 것이다.

In [20]:
import re
m = re.search('맛.{0,5}있.{0,5}[없않]', '안녕하세요. 맛이 있진 않다.. 다만 아쉬운 것은 포장지가 없다.')
print(m)
print('맛있다는 뜻' if m == None else '맛 없다는 뜻\n원문: ' + m.group())

m = re.search('맛.{0,5}있.{0,5}[없않]', '안녕하세요. 맛이 있다. 다만 아쉬운 것은 포장지가 없다.')
print(m)
print('맛있다는 뜻' if m == None else '맛 없다는 뜻\n원문: ' + m.group())


<_sre.SRE_Match object; span=(7, 14), match='맛이 있진 않'>
맛 없다는 뜻
원문: 맛이 있진 않
None
맛있다는 뜻
